In [22]:
import requests
from bs4 import BeautifulSoup
import re

def get_url(url='https://tiki.vn/laptop-may-vi-tinh-linh-kien/c1846?src=c.1846.hamburger_menu_fly_out_banner&_lc=Vk4wMzkwMTIwMDI%3D&is_cross_border=1'):
    """Get parsed HTML from url
      Input: url to the webpage
      Output: Parsed HTML text of the webpage
    """
    # Send Get request
    r = requests.get(url)

    # Parse HTML text
    soup = BeautifulSoup(r.text, 'html.parser')
    
    return soup

soup = get_url()

#get the text in the product-box-list class
product_list = soup.find('div', {'class':'product-box-list'})

#get the first product or first a
#all_product = product_list.find_all('a')
first_product = product_list.find('div', {'class':'product-item'})
first_product_category = first_product['data-category']
print(first_product_category)
first_product_title = first_product['data-title']
print(first_product_title)
first_product_final_price_sale_tag = first_product.find('p', {'class':'price-sale'}).text
prices = set(first_product_final_price_sale_tag.split())
print(prices)

if first_product.find('p', {'class':'installment'}):
    first_product_installment = first_product.find('p', {'class':'installment'}).text
print(first_product_installment)

first_product_review = first_product.find('p', {'class':'review'}).text
print(first_product_review)
first_product_rating = first_product.find('span', {'class':'rating-content'})
print(first_product_rating.span['style'])
percent_star_full = first_product_rating.span['style']
percent_star_in_number = percent_star_full[-3:-1]
print(percent_star_in_number)
#tiki now

#giao hang nuoc ngoai
if first_product.find('div', {'class':'ship-label-wrapper'}):
    first_product_ship_label = first_product.find('div', {'class':'ship-label-wrapper'}).text
print(first_product_ship_label)

#image
first_product_image = first_product.find('img', {'class', 'product-image img-responsive'})['src']
print(first_product_image)

#function tinh so sao
#get rid of hang chinh hang
# loops through page
#sleeps


Hàng Quốc Tế/Laptop & Máy Vi Tính/Màn hình máy tính
Giá Đỡ Màn Hình Brateck LDT12-C011
{'1.017.000đ', '498.000đ', '-51%'}

Trả góp 0%

(10 nhận xét)
width:98%
98



                        Hàng quốc tế
                    

https://salt.tikicdn.com/cache/280x280/ts/product/45/04/08/2d8f6ed4b049057ee24fa586508b3235.jpg
